In [55]:
!pip install backtrader
!pip install yfinance

In [56]:
import backtrader as bt
import math
import yfinance as yf

추세전략

In [57]:
class BollingerBands(bt.Strategy):
  params = dict(
      bb_period=20, # 볼린저 밴드 기간
      bb_devfactor=1, # 볼린저 밴드 표준편차
      ma_period=50 # 이동평균선 기간
  )

  def __init__(self):
    # 볼린저밴드
    self.bb = bt.indicators.BollingerBands(self.data.close, period=self.params.bb_period, devfactor=self.params.bb_devfactor)
    # 이동평균선
    self.ma = bt.indicators.MovingAverageSimple(self.data.close, period=self.params.ma_period)
    # 매수 신호 추적 변수
    self.order = None

  def next(self):
    if self.order:  # 만약 주문이 있으면 대기
      return
    # 포지션이 없을 때(시장 미진입 상태)
    if not self.position:
      # 매수 : 종가가 볼린저 밴드 상단선을 돌파
      if self.data.close[0] > self.bb.lines.top[0]:
        order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
        self.buy(size=order_size)

    # 포지션이 있을 때(시장 진입 상태)
    else:
      # 종가가 이평선 하향 돌파 -> 청산
      if self.data.close[0] < self.ma[0]:
        self.close()


  def log(self, message):
    print(message)

  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
      # Buy/Sell order submitted/accepted to/by broker - Nothing to do
      return

    # Check if an order has been completed
    # Attention: broker could reject order if not enough cash
    cur_date = None
    if order.status in [order.Completed]:
      cur_date = order.data.datetime.date(0)
      if order.isbuy():
        self.log(
            f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      elif order.issell():
        self.log(
            f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
      self.log(
          f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')

In [68]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(BollingerBands)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-01')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')


[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2020-04-02 [매수 주문 실행] 종목:  	 수량: 1498 	 가격: 6606.78
2020-04-03 [매도 주문 실행] 종목:  	 수량: -1498 	 가격: 6797.40
2020-04-04 [매수 주문 실행] 종목:  	 수량: 1503 	 가격: 6738.38
2020-04-05 [매도 주문 실행] 종목:  	 수량: -1503 	 가격: 6862.54
2020-04-07 [매수 주문 실행] 종목:  	 수량: 1409 	 가격: 7273.64
2020-04-08 [매도 주문 실행] 종목:  	 수량: -1409 	 가격: 7179.28
2020-04-09 [매수 주문 실행] 종목:  	 수량: 1370 	 가격: 7337.97
2020-04-10 [매도 주문 실행] 종목:  	 수량: -1370 	 가격: 7303.82
2020-04-19 [매수 주문 실행] 종목:  	 수량: 1370 	 가격: 7260.92
2020-06-20 [매도 주문 실행] 종목:  	 수량: -1370 	 가격: 9290.96
2020-07-09 [매수 주문 실행] 종목:  	 수량: 1340 	 가격: 9427.99
2020-07-10 [매도 주문 실행] 종목:  	 수량: -1340 	 가격: 9273.36
2020-07-22 [매수 주문 실행] 종목:  	 수량: 1317 	 가격: 9375.08
2020-09-04 [매도 주문 실행] 종목:  	 수량: -1317 	 가격: 10230.37
2020-10-09 [매수 주문 실행] 종목:  	 수량: 1227 	 가격: 10927.91
2021-04-19 [매도 주문 실행] 종목:  	 수량: -1227 	 가격: 56191.59
2021-05-09 [매수 주문 실행] 종목:  	 수량: 1158 	 가격: 58877.39
2021-05-11 [매도 주문 실행] 종목:  	 수량: -1158 	 가격: 55847.24
2021-06-14 [매수 주문 실행] 종목:  	 수량: 1643 	 가격: 39016.

스탑로스 10% 설정

In [59]:
class BollingerBandsStopLoss(bt.Strategy):
  params = dict(
      bb_period=20, # 볼린저 밴드 기간
      bb_devfactor=1, # 볼린저 밴드 표준편차
      ma_period=50, # 이동평균선 기간
      stop_loss=0.1, # 손절 비율(10%)
  )

  def __init__(self):
    # 볼린저밴드
    self.bb = bt.indicators.BollingerBands(self.data.close, period=self.params.bb_period, devfactor=self.params.bb_devfactor)
    # 이동평균선
    self.ma = bt.indicators.MovingAverageSimple(self.data.close, period=self.params.ma_period)
    # 매수 신호 추적 변수
    self.order = None
    # 매수 가격 저장
    self.buy_price = None

  def next(self):
    if self.order:  # 만약 주문이 있으면 대기
      return
    # 포지션이 없을 때(시장 미진입 상태)
    if not self.position:
      # 매수 : 종가가 볼린저 밴드 상단선을 돌파
      if self.data.close[0] > self.bb.lines.top[0]:
        order_size = math.floor(self.broker.get_value() / self.datas[0].close * 0.99)
        self.buy(size=order_size)
        self.buy_price = self.data.close[0]

    # 포지션이 있을 때(시장 진입 상태)
    else:
      # 종가가 이평선 하향 돌파 or 매수가 대비 10% 하락 -> 청산
      stop_loss_price = self.buy_price * (1 - self.params.stop_loss) # 손절가 계산
      if self.data.close[0] < stop_loss_price or self.data.close[0] < self.ma[0]:
        self.close()
        self.buy_price = None # 매도 후 매수 가격 초기화


  def log(self, message):
    print(message)

  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
      # Buy/Sell order submitted/accepted to/by broker - Nothing to do
      return

    # Check if an order has been completed
    # Attention: broker could reject order if not enough cash
    cur_date = None
    if order.status in [order.Completed]:
      cur_date = order.data.datetime.date(0)
      if order.isbuy():
        self.log(
            f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      elif order.issell():
        self.log(
            f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
      self.log(
          f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}')


In [60]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(BollingerBandsStopLoss)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-01')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

Starting Portfolio Value: 10000000.00


[*********************100%***********************]  1 of 1 completed


2020-04-02 [매수 주문 실행] 종목:  	 수량: 1498 	 가격: 6606.78
2020-04-03 [매도 주문 실행] 종목:  	 수량: -1498 	 가격: 6797.40
2020-04-04 [매수 주문 실행] 종목:  	 수량: 1503 	 가격: 6738.38
2020-04-05 [매도 주문 실행] 종목:  	 수량: -1503 	 가격: 6862.54
2020-04-07 [매수 주문 실행] 종목:  	 수량: 1409 	 가격: 7273.64
2020-04-08 [매도 주문 실행] 종목:  	 수량: -1409 	 가격: 7179.28
2020-04-09 [매수 주문 실행] 종목:  	 수량: 1370 	 가격: 7337.97
2020-04-10 [매도 주문 실행] 종목:  	 수량: -1370 	 가격: 7303.82
2020-04-19 [매수 주문 실행] 종목:  	 수량: 1370 	 가격: 7260.92
2020-06-20 [매도 주문 실행] 종목:  	 수량: -1370 	 가격: 9290.96
2020-07-09 [매수 주문 실행] 종목:  	 수량: 1340 	 가격: 9427.99
2020-07-10 [매도 주문 실행] 종목:  	 수량: -1340 	 가격: 9273.36
2020-07-22 [매수 주문 실행] 종목:  	 수량: 1317 	 가격: 9375.08
2020-09-04 [매도 주문 실행] 종목:  	 수량: -1317 	 가격: 10230.37
2020-10-09 [매수 주문 실행] 종목:  	 수량: 1227 	 가격: 10927.91
2021-04-19 [매도 주문 실행] 종목:  	 수량: -1227 	 가격: 56191.59
2021-05-09 [매수 주문 실행] 종목:  	 수량: 1158 	 가격: 58877.39
2021-05-11 [매도 주문 실행] 종목:  	 수량: -1158 	 가격: 55847.24
2021-06-14 [매수 주문 실행] 종목:  	 수량: 1643 	 가격: 39016.

추세전략이라 그런지 스탑로스가 있어도 맨 처음것과 동일

+10%마다 1/5씩 분할 매수

In [77]:
import backtrader as bt
import math

class BollingerBandsPartialBuy(bt.Strategy):
    params = dict(
        bb_period=20,             # 볼린저 밴드 기간
        bb_devfactor=2,           # 볼린저 밴드 표준편차
        ma_period=50,             # 이동평균선 기간
        max_additional_buys=5,    # 최대 분할 매수 횟수
        drop_trigger=0.10,        # 진입가 대비 10% 상승 시마다 추가 매수
    )

    def __init__(self):
        # 볼린저밴드
        self.bb = bt.indicators.BollingerBands(
            self.data.close,
            period=self.params.bb_period,
            devfactor=self.params.bb_devfactor
        )
        # 이동평균선 (추세 확인)
        self.ma = bt.indicators.MovingAverageSimple(
            self.data.close,
            period=self.params.ma_period
        )
        self.order = None

        self.buy_price = None             # 현재까지의 '평균 단가'
        self.num_additional_buys = 0      # 분할 매수 횟수 (최대 5회까지)

    def next(self):
        # 주문 진행 중이면 대기
        if self.order:
            return

        if not self.position:
            # 볼린저 상단 돌파 시 매수(예시)
            if self.data.close[0] > self.bb.top[0]:
                order_size = math.floor(self.broker.get_value() / self.data.close[0] * 0.2)
                self.order = self.buy(size=order_size)
                # buy_price는 notify_order에서 체결가를 받아온 뒤 설정
                self.num_additional_buys = 0

        # 포지션이 있는 경우
        else:
            # 추가 매수 로직
            if self.buy_price is not None:
                current_price = self.data.close[0]
                drop_ratio = (current_price - self.buy_price) / self.buy_price

                if drop_ratio >= self.params.drop_trigger and self.num_additional_buys < self.params.max_additional_buys:
                    add_size = math.floor(self.broker.get_value() / current_price * 0.2)
                    self.order = self.buy(size=add_size)
                    # 체결 후 notify_order에서 평균단가를 갱신
                    # 추가 매수 횟수는 체결 후에 += 1로 처리

            # 청산 조건
            if self.data.close[0] < self.ma[0]:
                self.order = self.close()
                self.buy_price = None


    def log(self, txt):
        dt = self.data.datetime.date(0)
        print(f'{dt} - {txt}')

    def notify_order(self, order):
        """
        주문 체결 이후의 평균단가 갱신 로직
        """
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            dt = order.data.datetime.date(0)

            if order.isbuy():
                # 1) 체결 정보
                fill_price = order.executed.price   # 체결가
                fill_size = order.executed.size     # 체결 수량 (양수)
                self.log(f'[매수 체결] 가격:{fill_price:.2f}, 수량:{fill_size:.4f}')

                # 2) 기존 포지션 수량 (체결 완료 후 포지션에 반영됨)
                #    order가 체결된 직후, backtrader는 포지션을 업데이트하므로
                current_position_size = self.position.size

                # 3) 평균단가 갱신
                if self.buy_price is None:
                    # 최초 매수인 경우
                    self.buy_price = fill_price
                else:
                    # 추가 매수인 경우
                    # (참고) 새로 반영된 self.position.size = 이전 size + fill_size
                    # 우리가 알고 싶은 것은 "체결 직전 size"이므로:
                    old_position_size = current_position_size - fill_size

                    if old_position_size <= 0:
                        # 혹시 포지션이 0이었거나 음수였다가(숏 포지션이었다가)
                        # 이번에 매수로 롱 포지션이 됐다면 그냥 fill_price로 셋팅
                        self.buy_price = fill_price
                    else:
                        # 평균단가 = (기존포지션 * 기존평단 + 신규수량 * 체결가) / (총수량)
                        new_average = (
                            (old_position_size * self.buy_price) + (fill_size * fill_price)
                        ) / (old_position_size + fill_size)

                        self.buy_price = new_average

                # 추가 매수 횟수 증가
                if self.num_additional_buys < self.params.max_additional_buys:
                    # 최초 매수 시에는 num_additional_buys==0 -> 매수 체결이 "추가"는 아님
                    # 다만 첫 매수 후도 "추가 매수"로 셀 수도 있어 로직을 원하는 대로 수정
                    self.num_additional_buys += 1

            elif order.issell():
                fill_price = order.executed.price
                fill_size = order.executed.size
                self.log(f'[매도 체결] 가격:{fill_price:.2f}, 수량:{fill_size:.4f}')
                # 매도 후 포지션이 0이면 buy_price 초기화
                if self.position.size == 0:
                    self.buy_price = None
                    self.num_additional_buys = 0

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('주문 취소/마진/거부')

        self.order = None

    def notify_trade(self, trade):
        """
        거래가 완전히 종료될 때(포지션=0) PnL 로그를 확인하고 싶다면 추가
        """
        if trade.isclosed:
            self.log(f'거래 종료 - 손익: {trade.pnl:.2f}, 수수료포함: {trade.pnlcomm:.2f}')




In [78]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(BollingerBandsPartialBuy)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-01')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2020-04-25 - [매수 체결] 가격:7550.48, 수량:264.0000
2020-04-30 - [매수 체결] 가격:8797.67, 수량:234.0000
2020-05-03 - [매수 체결] 가격:8983.61, 수량:231.0000
2020-05-07 - [매수 체결] 가격:9261.90, 수량:228.0000
2020-05-08 - 주문 취소/마진/거부
2020-05-09 - 주문 취소/마진/거부
2020-05-10 - 주문 취소/마진/거부
2020-05-15 - 주문 취소/마진/거부
2020-05-18 - 주문 취소/마진/거부
2020-05-19 - 주문 취소/마진/거부
2020-05-20 - 주문 취소/마진/거부
2020-05-21 - 주문 취소/마진/거부
2020-05-29 - 주문 취소/마진/거부
2020-05-31 - 주문 취소/마진/거부
2020-06-02 - 주문 취소/마진/거부
2020-06-03 - 주문 취소/마진/거부
2020-06-04 - 주문 취소/마진/거부
2020-06-05 - 주문 취소/마진/거부
2020-06-06 - 주문 취소/마진/거부
2020-06-07 - 주문 취소/마진/거부
2020-06-08 - 주문 취소/마진/거부
2020-06-09 - 주문 취소/마진/거부
2020-06-10 - 주문 취소/마진/거부
2020-06-11 - 주문 취소/마진/거부
2020-06-13 - 주문 취소/마진/거부
2020-06-14 - 주문 취소/마진/거부
2020-06-17 - 주문 취소/마진/거부
2020-06-18 - 주문 취소/마진/거부
2020-06-20 - [매도 체결] 가격:9290.96, 수량:-957.0000
2020-06-20 - 거래 종료 - 손익: 652539.36, 수수료포함: 601148.29
2020-07-23 - [매수 체결] 가격:9527.14, 수량:222.0000
2020-07-28 - [매수 체결] 가격:11017.46, 수량:198.0000
2020-08-01 - [매수 체결] 가격:11322.

뭔가 추가적으로 설정하는 것보다 가장 단순한게 성능이 가장 잘 나왔다